# VC08 - Métodos de aprendizaje semi-supervisado



En esta práctica vamos a ver dos ejemplos: el primero, cómo funciona Naive Bayes con datos sin etiquetar, y el segundo, cómo podemos mejorar la versión básica del Naive Bayes aprendiendo un mejor modelo iterativamente mediante el algoritmo de EM.

## Naive Bayes SIN datos sin etiquetar

Realizamos los imports necesarios:

In [ ]:
# Import packages and libraries
import numpy as np
import random as rnd

from scipy.sparse import csr_matrix
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [8, 8]

Cargamos nuestros datos:

In [ ]:
# Load train and test data set with class labels 
train_Xy = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
test_Xy = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

Utilizamos la transformación tf-Idf para convertir las palabras a una matriz de números:

In [ ]:
# Convert all text data into tf-idf vectors
vectorizer = TfidfVectorizer(stop_words='english', min_df=3, max_df=0.9)
train_vec = vectorizer.fit_transform(train_Xy.data)
test_vec = vectorizer.transform(test_Xy.data)

In [ ]:
test_vec.shape

Dividimos los datos entre etiquetados y no etiquetados:

In [ ]:
# Divide train data set into labeled and unlabeled data sets
n_train_data = train_vec.shape[0]
split_ratio = 0.2 # labeled vs unlabeled
X_l, X_u, y_l, y_u = train_test_split(train_vec, train_Xy.target, train_size=split_ratio, random_state=42)
print(f'Número de instancias CON etiquetas: {X_l.shape}')
print(f'Número de instancias SIN etiquetas: {X_u.shape}')

Entrenamos nuestro BN con únicamente los datos etiquetados:

In [ ]:
nb_clf = # Tu código aquí ##
nb_clf.fit(X_l, y_l)

Y evaluamos su comportamiento:

In [ ]:
# Evaluamos comportamiento NB
pred_nb = nb_clf.predict(test_vec)
print(metrics.classification_report(test_Xy.target, pred_nb, target_names=test_Xy.target_names))
print(f'Accuracy score={metrics.accuracy_score(test_Xy.target, pred_nb)}')

In [ ]:
# Matriz de confusión
plt.figure(figsize=(12,12))
sns.set()
sns.heatmap(metrics.confusion_matrix(test_Xy.target, pred_nb), annot=True, fmt="d", linewidths=.5)
plt.title("Matriz de Confusion")

## EM con NaiveBayes

Nos descargamos la clase que implementa el algoritmo de EM:

In [ ]:
!wget https://raw.githubusercontent.com/jerry-shijieli/Text_Classification_Using_EM_And_Semisupervied_Learning/master/code/Semi_EM_NB.py

La importamos:

In [ ]:
from Semi_EM_NB import Semi_EM_MultinomialNB

Entrenamos nuestro NB con EM usando los datos etiquetados y los no etiquetados:

In [ ]:
X_l.shape

In [ ]:
X_u.shape

In [ ]:
test_Xy.target.shape

In [ ]:
em_nb_clf = ## Tu código aquí ##
em_nb_clf.fit(X_l, y_l, X_u)

Y lo evaluamos:

In [ ]:
# Evaluamos su comportamiento
pred_em_nb = em_nb_clf.predict(test_vec)
print(metrics.classification_report(test_Xy.target, pred_em_nb, target_names=test_Xy.target_names))
print(f'Accuracy score={metrics.accuracy_score(test_Xy.target, pred_em_nb)}')

In [ ]:
# plot confusion matrix for test dataset
plt.figure(figsize=(12,12))
sns.set()
sns.heatmap(metrics.confusion_matrix(test_Xy.target, pred_em_nb), annot=True, fmt="d", linewidths=.5)
plt.title("Matriz de Confusion")

Veamos las distribuciones aprendidas:

In [ ]:
print('NB: ', nb_clf.class_log_prior_)
print('EM NB: ', em_nb_clf.clf.class_log_prior_)

Y para ver de una forma más visual la mejora que ha habido, veamos la diferencia entre las matrices de confusión:

In [ ]:
# plot diferencias entre las matrices de confusion
plt.figure(figsize=(12,12))
sns.set()
sns.heatmap(metrics.confusion_matrix(test_Xy.target, pred_em_nb) - metrics.confusion_matrix(test_Xy.target, pred_nb), annot=True, fmt="d", linewidths=.5)
plt.title("Diferencias entre NB y EM NB")

FUENTE: https://github.com/jerry-shijieli/Text_Classification_Using_EM_And_Semisupervied_Learning/

- Explicaciones de NB bastante didácticas y útiles: 
   - https://towardsdatascience.com/all-about-naive-bayes-8e13cef044cf
   - https://medium.com/@srishtisawla/introduction-to-naive-bayes-for-classification-baefefb43a2d
- Uso de NB para sentiment analysis con código y muy didáctico: https://levelup.gitconnected.com/movie-review-sentiment-analysis-with-naive-bayes-machine-learning-from-scratch-part-v-7bb869391bab